<a href="https://colab.research.google.com/github/otabeksheyh/deep_learning/blob/main/3_or_7_DL_fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install fastai==2.5.3



In [ ]:
import torch
import fastai

# versiyalarni tekshirish
print(torch.__version__)
print(fastai.__version__)

In [ ]:
from fastai.vision.all import*
 #path
path = untar_data(URLs.MNIST_SAMPLE)
#path.ls()
#(path/'train'/'3').ls()
threes = (path/'valid'/'3').ls().sorted()
sevens = (path/'valid'/'7').ls().sorted()

In [ ]:
img3_sample = threes[1]
img3_sample

In [ ]:
img3 = Image.open(img3_sample)
img3

In [ ]:
img7_sample = sevens[2]
img7 = Image.open(img7_sample)
img7

In [ ]:

np.array(img3)[4:10, 4:10]

In [ ]:
tensor(img3)[4:10, 4:10]

In [ ]:
import pandas as pd
img3_t = tensor(img3)
df = pd.DataFrame(img3_t[4:22, 4:22])# ustun va qatorlarni ajratib oldik
#pandasni qulay tarafi : har bir dataframeni ortidagi bakgraundini ranggini ozgartira olamiz
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')


In [ ]:
#ML va DL dan foydalamasdan 3 va 7 ni aniqlaydigan datsur tuib koramiz 

three_tensors = [tensor(Image.open(i)) for i in threes]
seven_tensors = [tensor(Image.open(i)) for i in sevens]
len(seven_tensors), len(three_tensors)

In [ ]:
show_image(three_tensors[2])

In [ ]:
#ideal 3 va 7 ni topish uchun vektorlashgan hisoblashda aniqlaymiz. ustma ust yani -->( stacking )
# odatda kompyuter vision mummolarda 0 dan 1 gacha bolgan oraliqqa keltish talab qilinadi
#tensorlar qiymatini float() ga ozgartirib /255 ga bolsak o va 1 oraligiga tushib qoladi

stacked_threes = torch.stack(three_tensors).float()/255
stacked_sevens = torch.stack(seven_tensors).float()/255
stacked_sevens.shape


In [ ]:
mean3 = stacked_threes.mean(0)
show_image(mean3)

In [ ]:
mean7 = stacked_sevens.mean(0)
show_image(mean7)

In [ ]:
# xatolik olchash: MAE yoki L1 norm
#xatolik olchash: RMSE yoki L2 norm

s_3  = stacked_threes[1]
show_image(s_3)

In [ ]:
# MAE
mae = (s_3-mean3).abs().mean()

#RMSE
rmse = ((s_3-mean3)**2).mean().sqrt()
mae, rmse

3 ni 7 bn solishtirib koramiz


In [ ]:
# MAE
mae = (s_3-mean7).abs().mean()

#RMSE
rmse = ((s_3-mean7)**2).mean().sqrt()
mae, rmse

modelni aniqligini **broadcasting** orqali aniqlash

In [ ]:
valid_3_tens = torch.stack([tensor(Image.open(i))for i in (path/'valid'/'3').ls()]) #ls() orqali listga aylantirib olamiz
valid_3_tens = valid_3_tens.float()/225 # 0 va 1 oraligiga tushirib olamiz


valid_7_tens = torch.stack([tensor(Image.open(i))for i in (path/'valid'/'7').ls()]) #ls() orqali listga aylantirib olamiz
valid_7_tens = valid_7_tens.float()/225 # 0 va 1 oraligiga tushirib olamiz


valid_3_tens.shape, valid_7_tens.shape

**MAE** ni hisoblashni funksiyaga aylantiramiz

In [ ]:
def mae(a,b):
  return(a-b).abs().mean((-1, -2)) # -1,-2 bular axis yani o'qlar, buni qoyishdan sabab koplab rasmlar uchun ishlatiladi, 

#tekshirish
mae(s_3, mean3)

valid_3_errors = mae(valid_3_tens, mean3)
valid_7_errors = mae(valid_7_tens, mean7)

valid_3_errors, valid_3_errors.shape

**Broadcasting**

In [ ]:
def is_3(x):
  return mae(x, mean3)<mae(x, mean7)

def is_7(x):
  return mae(x, mean7)<mae(x, mean3)

is_3(s_3), is_3(s_3).float()

In [ ]:
accuracy_3s = is_3(valid_3_tens).float().mean()
accuracy_7s = is_7(valid_7_tens).float().mean()

accuracy_3s, accuracy_7s, (accuracy_3s+accuracy_7s)/2